In [1]:
import pandas as pd
import numpy as np

In [2]:
raw = pd.read_excel(
    '20240418 NumberFire.xlsx', sheet_name='past',
    converters={'Salary': lambda x: float(x.replace('$','').replace(',',''))}
)


raw.shape

(24244, 17)

In [3]:
# ranked = raw.copy()
ranked = raw[~raw['Salary'].isna()].copy()
ranked['dummy'] = 1
ranked['rank'] = ranked.groupby('url_suffix')['dummy'].cumsum() - 1

ranked

,Date,OPP,MIN,PTS,FGM-A,3PM-A,FTM-A,REB,AST,STL,BLK,TOV,PF,Salary,FP,Value,url_suffix,dummy,rank
0,4/10/24,ORL,15.52,2,0-3,0-2,2-2,0,0,1,0,1,1,3800.0,4.0,1.05,a-j-green,1,0
1,4/9/24,BOS,18.72,6,2-3,2-3,0-0,1,0,0,0,0,0,3800.0,7.2,1.89,a-j-green,1,1
2,4/7/24,NY,13.47,2,1-3,0-2,0-0,0,0,1,0,0,1,3800.0,5.0,1.32,a-j-green,1,2
3,4/5/24,TOR,22.67,6,2-5,2-5,0-0,3,0,0,0,1,2,3800.0,8.6,2.26,a-j-green,1,3
4,4/3/24,MEM,21.02,9,3-7,3-6,0-1,4,2,1,1,1,0,3600.0,21.8,6.06,a-j-green,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24239,11/4/23,ATL,32.30,25,9-16,0-0,7-10,5,3,0,0,0,2,7800.0,35.5,4.55,zion-williamson,1,63
24240,11/1/23,@ OKC,32.85,20,7-20,0-0,6-9,10,8,0,0,4,4,7800.0,40.0,5.13,zion-williamson,1,64
24241,10/30/23,GS,29.93,19,7-15,0-1,5-9,5,3,2,0,5,5,8100.0,30.5,3.77,zion-williamson,1,65
24242,10/28/23,NY,28.08,24,12-17,0-0,0-2,5,2,1,0,0,0,8000.0,36.0,4.50,zion-williamson,1,66


In [4]:
def build_features(df, N_lookback=10):
    data = []
    players = df['url_suffix'].unique()

    for j, player in enumerate(players):
        # set player-level data
        ply_df = df[df['url_suffix']==player]
        ply_dat = []
        m = ply_df.shape[0]

        # not enough games played, continue to next player
        if m < N_lookback+1:
            continue
        
        # i indexes games to predict
        # i+1 - i+N are games used for features
        for i in range(0, m-N_lookback):
            # FP is value to predict, Date and Salary are known before contest
            sections = [ply_df.loc[ply_df['rank']==i, ['url_suffix', 'Date', 'FP', 'Salary']]]

            # filter to last N_lookback games to create features
            mask = (ply_df['rank']>i) & (ply_df['rank']<=i+N_lookback)
            temp = ply_df[mask]

            # col lists features to be calculate
            # create pivot table of last N_lookback values
            for col in ['FP', 'MIN', 'Value', 'Salary']:
                col_section = temp.pivot_table(
                    index='url_suffix',
                    values=col,
                    columns='rank',
                    aggfunc='sum'
                ).astype(float).fillna(0)

                # rename columns and match index
                col_section.columns = [col + '_' + str(i) for i in range(1,N_lookback+1)]
                col_section.index = sections[0].index
                sections.append(col_section)

            # concatenate dataframes to create a record
            ply_dat.append(pd.concat(sections, axis='columns'))
        
        # concatenate records for player
        data.append(pd.concat(ply_dat, axis='rows'))

        # status bar
        print(f"{(j+1)/len(players):6.1%} completed\r", end="")

    # concatenate all records
    return pd.concat(data, axis='rows')

In [5]:
%time all_dat = build_features(ranked)

CPU times: total: 3min 10s
Wall time: 5min 7s


In [9]:
# all_dat.to_csv('features_lookback_10.csv', index=False)